In [48]:
from spotify_infos import sp, me

Ajouter une playlist

In [49]:
playlist = sp.user_playlist_create(
    user=me["id"],
    name="Mood2Music - Demo",
    public=False,
    description="Playlist generated during hackathon"
)
print("Playlist:", playlist["external_urls"]["spotify"])

Playlist: https://open.spotify.com/playlist/6d3z6sHGUc9T2LgqOuqxwb


In [50]:
print(playlist["id"])

6d3z6sHGUc9T2LgqOuqxwb


In [51]:
print(playlist)

{'collaborative': False, 'description': 'Playlist generated during hackathon', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6d3z6sHGUc9T2LgqOuqxwb'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/6d3z6sHGUc9T2LgqOuqxwb', 'id': '6d3z6sHGUc9T2LgqOuqxwb', 'images': [], 'primary_color': None, 'name': 'Mood2Music - Demo', 'type': 'playlist', 'uri': 'spotify:playlist:6d3z6sHGUc9T2LgqOuqxwb', 'owner': {'href': 'https://api.spotify.com/v1/users/thibaultm2003', 'id': 'thibaultm2003', 'type': 'user', 'uri': 'spotify:user:thibaultm2003', 'display_name': None, 'external_urls': {'spotify': 'https://open.spotify.com/user/thibaultm2003'}}, 'public': False, 'snapshot_id': 'AAABgYYGOz0/tc/C1z/Ch8+6h+DqmXSL', 'tracks': {'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'href': 'https://api.spotify.com/v1/playlists/6d3z6sHGUc9T2LgqOuqxwb/tracks', 'total': 0, 'items': []}}


In [52]:
sp.playlist_add_items(playlist["id"], [mhd_id])

{'snapshot_id': 'AAAAAi3SaRhlQxx0uwO7hH6axpgqeFQg'}

Ajouter un son à une playlist

In [53]:
sp.playlist_add_items(playlist["id"], ["spotify:track:4uLU6hMCjMI75M1A2tKUQC"])
print("Added one track.")

Added one track.


In [54]:
# --- ajouter un titre précis à une playlist ---
def findsongID(query : str, type = 'track'):
    results = sp.search(q = query, limit=1, type = type)
    song_id = results[type + 's']['items'][0]['id']
    return song_id

def findplaylistID(query : str, type = 'playlist'):
    results = sp.current_user_playlists(limit=50)
    for pl in results["items"]:
        if query.lower() in pl["name"].lower():
            return pl["id"]
        
def add2playlist(query_song : str, query_playlist : str, type = 'track'):
    song_id, playlist_id = findsongID(query_song), findplaylistID(query_playlist)
    sp.playlist_add_items(playlist_id, [f"spotify:track:{song_id}"])
    return (f"I added {query_song} to your playlist {query_playlist} !")

In [59]:
def _find_song_id(query: str, market: Optional[str] = None) -> Optional[str]:
    res = sp.search(q=query, limit=1, type="track", market=market)
    items = res.get("tracks", {}).get("items", []) or []
    return items[0]["id"] if items else None


def _find_my_playlist_id(query: str) -> Optional[str]:
    res = sp.current_user_playlists(limit=50)
    for pl in res.get("items", []):
        name = (pl.get("name") or "").lower()
        if query.lower() in name and pl.get("owner", {}).get("id") == me["id"]:
            return pl["id"]
    return None

def add2playlist(song_query: str, playlist_query: str, allow_duplicates: bool = True, market: Optional[str] = "from_token") -> Dict[str, Any]:
    """
    Find best match for 'song_query' and add it to YOUR playlist whose name contains 'playlist_query'.
    Scopes: playlist-read-private + playlist-modify-private (or public)
    """
    song_id = _find_song_id(song_query, market=market)
    if not song_id:
        return {"status": "not_found", "entity": "track", "query": song_query}

    playlist_id = _find_my_playlist_id(playlist_query)
    if not playlist_id:
        return {"status": "not_found", "entity": "playlist", "query": playlist_query}

    track_uri = f"spotify:track:{song_id}"

    if not allow_duplicates:
        items = sp.playlist_items(playlist_id, limit=100).get("items", [])
        if any(track_uri == (it.get("track") or {}).get("uri") for it in items):
            return {"status": "skipped", "reason": "duplicate", "playlist_id": playlist_id, "track_uri": track_uri}

    sp.playlist_add_items(playlist_id, [track_uri])
    pl = sp.playlist(playlist_id, fields="name,external_urls.spotify")
    return {"status": "added", "playlist_id": playlist_id, "playlist_name": pl["name"], "playlist_url": pl["external_urls"]["spotify"], "track_uri": track_uri}


In [63]:
def add2playlist(song_query: str, playlist_query: str, allow_duplicates: bool = True, market: Optional[str] = "from_token") -> Dict[str, Any]:
    """
    Find best match for 'song_query' and add it to YOUR playlist whose name contains 'playlist_query'.
    Scopes: playlist-read-private + playlist-modify-private (or public)
    """
    def _find_song_id(query: str, market: Optional[str] = None) -> Optional[str]:
        res = sp.search(q=query, limit=1, type="track", market=market)
        items = res.get("tracks", {}).get("items", []) or []
        return items[0]["id"] if items else None


    def _find_my_playlist_id(query: str) -> Optional[str]:
        res = sp.current_user_playlists(limit=50)
        for pl in res.get("items", []):
            name = (pl.get("name") or "").lower()
            if query.lower() in name and pl.get("owner", {}).get("id") == me["id"]:
                return pl["id"]
        return None
    
    
    song_id = _find_song_id(song_query, market=market)
    if not song_id:
        return {"status": "not_found", "entity": "track", "query": song_query}

    playlist_id = _find_my_playlist_id(playlist_query)
    if not playlist_id:
        return {"status": "not_found", "entity": "playlist", "query": playlist_query}

    track_uri = f"spotify:track:{song_id}"

    if not allow_duplicates:
        items = sp.playlist_items(playlist_id, limit=100).get("items", [])
        if any(track_uri == (it.get("track") or {}).get("uri") for it in items):
            return {"status": "skipped", "reason": "duplicate", "playlist_id": playlist_id, "track_uri": track_uri}

    sp.playlist_add_items(playlist_id, [track_uri])
    pl = sp.playlist(playlist_id, fields="name,external_urls.spotify")
    return {"status": "added", "playlist_id": playlist_id, "playlist_name": pl["name"], "playlist_url": pl["external_urls"]["spotify"], "track_uri": track_uri}



In [64]:
add2playlist("moulaga", "La putain de ta mère")

{'status': 'added',
 'playlist_id': '3ssN4qLkzvVsDeEnb9wpDZ',
 'playlist_name': 'La putain de ta mère',
 'playlist_url': 'https://open.spotify.com/playlist/3ssN4qLkzvVsDeEnb9wpDZ',
 'track_uri': 'spotify:track:4CWcaGtaOyAqBDKIeZMUe0'}

In [30]:
findsongID("afrotrap la puissance")

'6wdI9sg2VdgjqSg0ozu9PK'

In [31]:
findplaylistID("Afrotrap MHD")

'6jjAbvUQC5RifW8CNAItTe'

In [35]:
sp.playlist_add_items("6jjAbvUQC5RifW8CNAItTe", ["spotify:track:6wdI9sg2VdgjqSg0ozu9PK"])

HTTP Error for POST to https://api.spotify.com/v1/playlists/6jjAbvUQC5RifW8CNAItTe/tracks with Params: {'position': None} returned 403 due to Forbidden


SpotifyException: http status: 403, code: -1 - https://api.spotify.com/v1/playlists/6jjAbvUQC5RifW8CNAItTe/tracks:
 Forbidden, reason: None

Trouver son

In [19]:
from requests import Session
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [21]:
def search_id(query : str, playlist_name : str, type = 'track'):
    results = sp.search(q = query, limit=1, type = type)
    id = results[type + 's']['items'][0]['id']
    return id


In [26]:
print(search_id("mhd afro trap", "mhd"))

mhd_id = search_id("mhd afro trap", "mhd")

6wdI9sg2VdgjqSg0ozu9PK


In [66]:
from flask import Flask
import time

app = Flask(__name__)

@app.route("/health")
def health():
    return {"status": "ok", "time": time.strftime("%Y-%m-%d %H:%M:%S")}


In [80]:
results = sp.search(q="nous sommes les patates serene soundscapes", type="track", limit=1)
track = results["tracks"]["items"][0]

print("Track:", track["name"], "by", track["artists"][0]["name"])
print("Preview URL:", track["preview_url"])

# Télécharger la preview si dispo
url = track["preview_url"]
if url:
    r = requests.get(url)
    with open("preview.mp3", "wb") as f:
        f.write(r.content)
    print("Preview saved as preview.mp3")
else:
    print("No preview available for this track.")

Track: Impardonnable by Damso
Preview URL: None
No preview available for this track.


In [69]:
from pydub import AudioSegment

audio = AudioSegment.from_file("preview.mp3")
faster = audio.speedup(playback_speed=1.25)
faster.export("preview_faster.mp3", format="mp3")



FileNotFoundError: [Errno 2] No such file or directory: 'preview.mp3'

In [ ]:
results = sp.search(q="nous sommes les patates serene soundscapes", type="track", limit=1)
track = results["tracks"]["items"][0]

print("Track:", track["name"], "by", track["artists"][0]["name"])
print("Preview URL:", track["preview_url"])

# Télécharger la preview si dispo
url = track["preview_url"]
if url:
    r = requests.get(url)
    with open("preview.mp3", "wb") as f:
        f.write(r.content)
    print("Preview saved as preview.mp3")
else:
    print("No preview available for this track.")

Track: Impardonnable by Damso
Preview URL: None
No preview available for this track.


In [82]:
# test_musicgen.py
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import soundfile as sf
import torch

# 1) Charge le modèle "small" (plus léger)
MODEL_ID = "facebook/musicgen-small"

print("Loading model…")
processor = AutoProcessor.from_pretrained(MODEL_ID)
model = MusicgenForConditionalGeneration.from_pretrained(MODEL_ID)
model.eval()
model.to("cpu")  # laisse en CPU pour un test simple

# 2) Ton prompt (décris le style que tu veux)
prompt = "a hardcore LA rap music with police sirens and laughs"

# 3) Prépare les entrées
inputs = processor(
    text=[prompt],
    padding=True,
    return_tensors="pt"
)

# 4) Génère ~8 à 10 secondes de son
#    max_new_tokens ~ durée (grossière). 256 ≈ 5s, 512 ≈ 10s (approximatif).
with torch.inference_mode():
    audio_values = model.generate(
        **inputs,
        do_sample=True,
        guidance_scale=3.0,      # un peu de “contrainte” sur le prompt
        max_new_tokens=512       # ↑ pour + long (512 ~ ~10s)
    )

# 5) Sauvegarde en WAV
#    audio_values shape: (batch, channels, samples)
wav = audio_values[0, 0].cpu().numpy()
sr = model.config.audio_encoder.sampling_rate  # ex: 32000 Hz
out_path = "musicgen_demo.wav"
sf.write(out_path, wav, sr)
print(f"Saved: {out_path}  |  sr={sr} Hz  |  seconds ~ {len(wav)/sr:.1f}")


Loading model…


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 4809.98it/s]


Saved: musicgen_demo.wav  |  sr=32000 Hz  |  seconds ~ 10.2


In [ ]:
import base64

with open("musicgen_demo.wav", "wb") as f:
    f.write(base64.b64decode(response["data_base64"]))

In [87]:
from google.cloud import storage

client = storage.Client()

buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [98]:
from dotenv import load_dotenv
import os

load_dotenv()

GCS_ID= os.getenv("GCS_CREDENTIALS")

python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10
python-dotenv could not parse statement starting at line 11
python-dotenv could not parse statement starting at line 12
python-dotenv could not parse statement starting at line 13
python-dotenv could not parse statement starting at line 14
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18


In [88]:
import io, base64
from google.cloud import storage
from datetime import timedelta

In [92]:

# -------- Cache modèle (Génération de musique)-----------
MODEL_ID = "facebook/musicgen-small"
_processor = None
_model = None

def _load_model_once():
    global _processor, _model
    if _processor is None or _model is None:
        _processor = AutoProcessor.from_pretrained(MODEL_ID)
        _model = MusicgenForConditionalGeneration.from_pretrained(MODEL_ID)
        _model.eval().to("cpu")


In [94]:
def upload_base64_to_gcs(b64_data: str, dest_name: str, content_type: str = "application/octet-stream", expires_sec: int = 3600) -> str:
    """
    Décode un base64 et l'upload dans GCS.
    Retourne une URL signée pour télécharger le fichier.
    """
    client = storage.Client(project = "My First Project")
    bucket = client.bucket("mood2music-uploads")
    blob = bucket.blob(dest_name)

    # décoder le base64 en bytes
    file_bytes = base64.b64decode(b64_data)

    # upload depuis mémoire
    blob.upload_from_string(file_bytes, content_type=content_type)

    # générer une URL signée (download direct)
    url = blob.generate_signed_url(
        version="v4",
        expiration=timedelta(seconds=expires_sec),
        method="GET"
    )
    return url

In [95]:
def generate_music(user_prompt: str, seconds: int = 10, guidance_scale: float = 3.0):
    """
    Génère un extrait musical avec MusicGen et renvoie un WAV encodé en base64.
    - user_prompt: description du style (ex: "lofi chill with warm piano")
    - seconds: durée approx (5-12s recommandé en CPU)
    - guidance_scale: 2.0–4.0 = contrainte du prompt
    """
    try:
        _load_model_once()

        # approx: ~256 tokens ≈ 5s
        max_new_tokens = max(128, min(1024, int(51.2 * seconds)))

        inputs = _processor(text=[user_prompt], padding=True, return_tensors="pt")
        with torch.inference_mode():
            audio_values = _model.generate(
                **inputs,
                do_sample=True,
                guidance_scale=guidance_scale,
                max_new_tokens=max_new_tokens
            )

        wav = audio_values[0, 0].cpu().numpy()
        sr = _model.config.audio_encoder.sampling_rate

        # Écrire dans un buffer mémoire
        buffer = io.BytesIO()
        sf.write(buffer, wav, sr, format="WAV")
        buffer.seek(0)

        # Encoder en base64
        b64_audio = base64.b64encode(buffer.read()).decode("utf-8")

        url = upload_base64_to_gcs(b64_audio, "test")
        print(url)
        return {
            "status": "ok",
            "prompt": user_prompt,
            "approx_seconds": round(len(wav) / sr, 2),
            "sampling_rate": sr,
            "filename": "musicgen_demo.wav",
            "mimetype": "audio/wav",
            "data_base64": b64_audio
        }
    
    

    except Exception as e:
        return {"status": "error", "message": str(e)}

In [96]:
generate_music("country music")

{'status': 'error',
 'message': 'Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.'}

In [103]:
import os
os.environ["GCS_BUCKET"] = "mood2music-uploads"

BUCKET_NAME = os.environ["GCS_BUCKET"]
print("Bucket choisi:", BUCKET_NAME)


Bucket choisi: mood2music-uploads


In [104]:
from google.cloud import storage
import os
from datetime import timedelta

# On lit le nom du bucket dans la variable d’env
BUCKET_NAME = os.environ["GCS_BUCKET"]

def upload_wav_to_gcs(local_path: str, dest_name: str) -> str:
    """
    Upload un fichier WAV dans GCS et retourne une URL signée (valable 1h).
    
    - local_path : chemin du fichier local (ex: "musicgen_demo.wav")
    - dest_name : chemin/nom dans le bucket (ex: "generated/musicgen_demo.wav")
    """
    # Crée un client GCS (il va utiliser ton .json via GOOGLE_APPLICATION_CREDENTIALS)
    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)
    blob = bucket.blob(dest_name)

    # Upload du fichier local
    blob.upload_from_filename(local_path, content_type="audio/wav")

    # Génère une URL signée pour télécharger
    url = blob.generate_signed_url(
        version="v4",
        expiration=timedelta(hours=1),
        method="GET"
    )
    return url


# Exemple d’utilisation
if __name__ == "__main__":
    local_file = "musicgen_demo.wav"  # ton fichier local
    gcs_name = "generated/musicgen_demo.wav"  # où le stocker dans le bucket

    url = upload_wav_to_gcs(local_file, gcs_name)
    print("✅ Fichier uploadé ! Télécharge-le ici :", url)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.